# Amazon Bedrock

Pre-requisites:
- AWS Credentials
- Bedrock model access to desire model
- setup on-demand inference for "Provisioned Throughput"; use **no commitment** ... the whole provisioning process may take more than 5min
> very important: remember to <u>delete immediately</u> after use (just in case)
> https://aws.amazon.com/bedrock/pricing/
- [supported models](https://docs.aws.amazon.com/bedrock/latest/userguide/batch-inference-supported.html) by region in AWS

If the "Provisioned Throughput" is not setup, the following typical error messages may appear in your traceback:


```
ValidationException: An error occurred (ValidationException) when calling the InvokeModel operation: Invocation of model ID anthropic.claude-3-5-haiku-20241022-v1:0 with on-demand throughput isn’t supported. Retry your request with the ID or ARN of an inference profile that contains this model.
```

In [10]:
!pip install langchain

In [1]:
!pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 2.6 MB/s eta 0:00:00


In [7]:
import os
from google.colab import userdata

os.environ['AWS_ACCESS_KEY_ID'] = userdata.get('AWS_ACCESS_KEY_ID')
os.environ['AWS_SECRET_ACCESS_KEY'] = userdata.get('AWS_SECRET_ACCESS_KEY')
os.environ['AWS_REGION'] = userdata.get('AWS_REGION')
os.environ['AWS_ACCOUNT_ID'] = userdata.get('AWS_ACCOUNT_ID')

In [15]:
import boto3
import json

bedrock_runtime = boto3.client(
    service_name='bedrock-runtime',
    region_name='us-east-1')

prompt = "What is the capital of France?"

# original intended for claude 3.5 haiku but this model is available for "Provisioned throughput"
# kwargs = {
#   "modelId": "anthropic.claude-3-5-haiku-20241022-v1:0",
#   "contentType": "application/json",
#   "accept": "application/json",
#   "body": json.dumps({
#     "anthropic_version": "bedrock-2023-05-31",
#     "max_tokens": 200,
#     "top_k": 250,
#     "stopSequences": [],
#     "temperature": 1,
#     "top_p": 0.999,
#     "messages": [
#       {
#         "role": "user",
#         "content": [
#           {
#             "type": "text",
#             "text": prompt
#           }
#         ]
#       }
#     ]
#   })
# }

kwargs = {
  "modelId": "anthropic.claude-3-haiku-20240307-v1:0",
  "contentType": "application/json",
  "accept": "application/json",
  "body": json.dumps({
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 1000,
    "messages": [
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": prompt
          }
        ]
      }
    ]
  })
}

response = bedrock_runtime.invoke_model(**kwargs)
response_body = json.loads(response.get('body').read())

from pprint import pprint
pprint(response_body)

{'content': [{'text': 'The capital of France is Paris.', 'type': 'text'}],
 'id': 'msg_bdrk_01QNq9FTaKcukdgBNeky9jEp',
 'model': 'claude-3-haiku-20240307',
 'role': 'assistant',
 'stop_reason': 'end_turn',
 'stop_sequence': None,
 'type': 'message',
 'usage': {'input_tokens': 14, 'output_tokens': 10}}


In [17]:
pprint(response_body['content'][0])

{'text': 'The capital of France is Paris.', 'type': 'text'}
